
# 完全に接続された層から畳み込みまで

:label: `sec_why-conv`

今日に至るまで、これまで説明してきたモデルは、表形式のデータを扱う場合に適切なオプションとして残っています。表形式とは、データが例に対応する行と特徴に対応する列で構成されていることを意味します。表形式のデータの場合、私たちが求めるパターンには特徴間の相互作用が含まれる可能性があると予想されるかもしれませんが、特徴がどのように相互作用するかについて*先験的に*構造を想定していません。

場合によっては、より巧妙なアーキテクチャの構築を導くための知識が本当に不足していることがあります。このような場合、MLP が私たちにできる最善の方法である可能性があります。ただし、高次元の知覚データの場合、そのような構造のないネットワークは扱いにくくなる可能性があります。

たとえば、猫と犬を区別する実行例に戻りましょう。データ収集において徹底的な仕事を行っており、1 メガピクセルの写真の注釈付きデータセットを収集しているとします。これは、ネットワークへの各入力に 100 万次元があることを意味します。 1,000 隠れ次元に積極的に削減する場合でも、$10^6 \times 10^3 = 10^9$ パラメーターによって特徴付けられる完全に接続された層が必要になります。大量の GPU、分散最適化の才能、そして並外れた忍耐力がない限り、このネットワークのパラメーターを学習することは不可能であることが判明する可能性があります。

注意深い読者は、1 メガピクセルの解像度は必要ないかもしれないという理由でこの議論に反対するかもしれません。ただし、10 万ピクセルで済むかもしれませんが、サイズ 1000 の隠れ層は、画像の適切な表現を学習するために必要な隠れユニットの数を大幅に過小評価しているため、実用的なシステムには依然として数十億のパラメーターが必要です。さらに、非常に多くのパラメータを当てはめて分類器を学習するには、膨大なデータセットを収集する必要がある場合があります。しかし、今日では人間もコンピューターも猫と犬を非常によく区別することができ、これらの直観とは一見矛盾しているようです。それは、画像が人間と機械学習モデルの両方で利用できる豊富な構造を示しているためです。畳み込みニューラル ネットワーク (CNN) は、自然画像内の既知の構造の一部を利用するために機械学習が取り入れている創造的な方法の 1 つです。

## 不変性

画像内のオブジェクトを検出したいと想像してください。物体を認識するためにどのような方法を使用する場合でも、画像内の物体の正確な位置を過度に気にすべきではないと思われます。理想的には、私たちのシステムはこの知識を活用する必要があります。通常、豚は飛べませんし、飛行機も通常は泳ぎません。それにもかかわらず、画像の上部に豚が表示されたことを認識する必要があります。ここでは、子供向けゲーム「ウォーリーを探せ」 ( :numref: `img_waldo`に描かれている) からインスピレーションを得ることができます。このゲームは、アクティビティが満載の多数のカオスなシーンで構成されています。ウォーリーはそれぞれのどこかに現れ、通常はありそうもない場所に潜んでいます。読者の目標は彼を見つけることです。彼の特徴的な服装にもかかわらず、気を散らすものが多数あるため、これは驚くほど難しい場合があります。ただし、*ウォーリーがどのように見えるかは、**ウォーリーがいる場所*には依存しません。 Waldo 検出器を使用して画像をスイープし、各パッチにスコアを割り当てて、パッチに Waldo が含まれる可能性を示すことができます。実際、多くのオブジェクト検出およびセグメンテーション アルゴリズムはこのアプローチに基づいています:cite: `Long.Shelhamer.Darrell.2015` 。 CNN はこの*空間不変性*の考え方を体系化し、それを利用してより少ないパラメータで有用な表現を学習します。 

![](../img/where-wally-walker-books.jpg) :幅: `400px` :ラベル: `img_waldo`

コンピューター ビジョンに適したニューラル ネットワーク アーキテクチャの設計の指針となるいくつかの要望を列挙することで、これらの直感をより具体的にすることができます。
1. 最初の層では、画像内のどこにパッチが現れるかに関係なく、ネットワークは同じパッチに同様に応答する必要があります。この原則は、*翻訳不変性*(または*翻訳等変性*) と呼ばれます。
1. ネットワークの最初の層は、離れた領域の画像の内容に関係なく、ローカルな領域に焦点を当てる必要があります。これが*地域性*原理です。最終的には、これらのローカル表現を集約して、画像全体のレベルで予測を行うことができます。
1. 私たちが進むにつれて、より深い層は、自然界の高レベルの視覚と同様の方法で、画像の長距離の特徴をキャプチャできるようになります。

これが数学にどのように変換されるかを見てみましょう。

##  MLP の制約

まず、入力として 2 次元画像 $\mathbf{X}$ と、同様に行列として表されるその直接の隠れ表現 $\mathbf{H}$ を持つ MLP を考えることができます (コードでは 2 次元テンソルです)。ここで、$\mathbf{X}$ と $\mathbf{H}$ はどちらも同じ形状です。それを理解しましょう。私たちは入力だけでなく、隠れた表現も空間構造を持つものと考えています。

 $[\mathbf{X}] *{i, j}$ と $[\mathbf{H}]* {i, j}$ が入力画像と隠れ表現の $(i,j)$ の位置にあるピクセルを表すものとします。それぞれ。したがって、各隠れユニットが各入力ピクセルから入力を受け取るようにするには、(MLP で以前に行ったように) 重み行列の使用からパラメータを 4 次の重みテンソルとして表現することに切り替えることになります $\mathsf{W}$ 。 $\mathbf{U}$ にバイアスが含まれていると仮定すると、全結合層を形式的に次のように表すことができます。

 $$\begin{aligned} \left[\mathbf{H}\right] *{i, j} &amp;= [\mathbf{U}]* {i, j} + \sum_k \sum_l[\mathsf{W}] *{ i, j, k, l} [\mathbf{X}]* {k, l}\ &amp;= [\mathbf{U}] *{i, j} + \sum_a \sum_b [\mathsf{V}]* {i, j, a, b} [\mathbf{X}]_{i+a, j+b}.\end{aligned}$$

 $\mathsf{W}$ から $\mathsf{V}$ への切り替えは、両方の 4 次テンソルの係数間に 1 対 1 の対応があるため、今のところ完全に表面的なものです。 $k = i+a$ および $l = j+b$ となるように、添え字 $(k, l)$ のインデックスを再作成するだけです。つまり、$[\mathsf{V}] *{i, j, a, b} = [\mathsf{W}]* {i, j, i+a, j+b}$ とします。インデックス $a$ と $b$ は、正と負の両方のオフセットにわたって実行され、イメージ全体をカバーします。隠れ表現 $[\mathbf{H} *] {i, j}$ 内の任意の位置 ($i$, $j$) について、$(i を中心とする $x$ 内のピクセルを合計することによってその値を計算します) , j)$ と $[\mathsf{V}] {i, j, a, b}$ で重み付けされます*。先に進む前に、このパラメータ化の*単一*レイヤーに必要なパラメータの合計数を考えてみましょう。$1000 \times 1000$ の画像 (1 メガピクセル) は、$1000 \times 1000$ の非表示表現にマッピングされます。これには $10^{12}$ パラメータが必要ですが、これは現在のコンピュータが処理できるパラメータをはるかに超えています。

### 翻訳の不変性

ここで、上で確立した最初の原則、翻訳不変性 :cite: `Zhang.ea.1988`を呼び出してみましょう。これは、入力 $\mathbf{X}$ のシフトは単に隠れ表現 $\mathbf{H}$ のシフトにつながるはずであることを意味します。これは、$\mathsf{V}$ と $\mathbf{U}$ が実際に $(i, j)$ に依存していない場合にのみ可能です。したがって、 $[\mathsf{V}] *{i, j, a, b} = [\mathbf{V}]* {a, b}$ があり、$\mathbf{U}$ は定数です。たとえば $う$。その結果、$\mathbf{H}$ の定義を簡素化できます。

 $$[\mathbf{H}] *{i, j} = u + \sum_a\sum_b [\mathbf{V}]* {a, b} [\mathbf{X}]_{i+a, j+b} .$$

これは*畳み込み*です!位置 $(i, j)$ の近傍にある $(i+a, j+b)$ のピクセルを係数 $[\mathbf{V}] *{a, b}$ で効果的に重み付けして、値 $ を取得します。 [\mathbf{H}]* {i, j}$. $[\mathbf{V}] *{a, b}$ は画像内の位置に依存しないため、$[\mathsf{V}] {i, j, a, b}$ よりも必要な係数がはるかに少ないことに*注意してください。 。その結果、必要なパラメータの数は $10^{12}$ ではなくなり、より合理的な $4 \cdot 10^6$ になります。つまり、$a, b \in (-1000, 1000)$ への依存関係がまだ残っています。一言で言えば、私たちは大きな進歩を遂げました。時間遅延ニューラル ネットワーク (TDNN) は、このアイデアを利用した最初の例の一部です (引用: `Waibel.Hanazawa.Hinton.ea.1989` )。

### 地域性

次に、2 番目の原則である局所性を取り上げてみましょう。上記の動機のように、$[\mathbf{H}] *{i, j で何が起こっているかを評価するために関連情報を収集するために、位置 $(i, j* )$ からそれほど遠くを見る必要はないと考えています。 *}$。これは、ある範囲外 $|a|&gt; \Delta$ または $|b| を意味します。 &gt; \Delta$ の場合、$[\mathbf{V}] {a, b} = 0$ を設定する必要があります*。同様に、$[\mathbf{H}]_{i, j}$ を次のように書き換えることができます。

 $$[\mathbf{H}] *{i, j} = u + \sum* {a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V }] *{a, b} [\mathbf{X}]* {i+a, j+b}.$$ :eqlabel: `eq_conv-layer`

これにより、パラメーターの数が $4 \cdot 10^6$ から $4 \Delta^2$ に減ります。ここで、$\Delta$ は通常 $10$ より小さくなります。そのため、パラメータの数をさらに 4 桁削減しました。 :eqref: `eq_conv-layer` 、簡単に言えば、*畳み込み層と*呼ばれるものであることに注意してください。*畳み込みニューラル ネットワーク*(CNN) は、畳み込み層を含む特別なニューラル ネットワークのファミリーです。深層学習研究コミュニティでは、 $\mathbf{V}$ は、*畳み込みカーネル*、*フィルター*、または単に学習可能なパラメーターである層の*重み*と呼ばれます。

以前は、画像処理ネットワーク内の 1 つのレイヤーを表現するのに数十億のパラメーターが必要だったかもしれませんが、現在では、入力または非表示の表現の次元を変更することなく、通常は数百のみで済みます。このパラメータの大幅な削減の代償として、機能が変換不変になり、各隠れアクティベーションの値を決定するときにレイヤーがローカル情報のみを組み込むことができるようになりました。すべての学習は、誘導バイアスを課すことに依存しています。そのバイアスが現実と一致すると、目に見えないデータまでよく一般化するサンプル効率の高いモデルが得られます。しかしもちろん、これらのバイアスが現実と一致しない場合、たとえば画像が変換不変ではないことが判明した場合、モデルはトレーニング データに適合することさえ困難になる可能性があります。

このパラメータの劇的な削減により、私たちは最後の要望にたどり着きます。つまり、より深い層は画像のより大きく複雑な側面を表現する必要があるということです。これは、非線形層と畳み込み層を繰り返しインターリーブすることで実現できます。

## 畳み込み

:eqref: `eq_conv-layer`が畳み込みと呼ばれる理由を簡単に確認してみましょう。数学では、2 つの関数 :cite: `Rudin.1973` 、たとえば $f, g: \mathbb{R}^d \to \mathbb{R}$ の*間の畳み込み*は次のように定義されます。

 $$(f * g)(\mathbf{x}) = \int f(\mathbf{z}) g(\mathbf{x}-\mathbf{z}) d\mathbf{z}.$$

つまり、1 つの関数が「反転」されて $\mathbf{x}$ だけシフトされたときの $f$ と $g$ の間の重なりを測定します。離散オブジェクトがある場合は常に、積分は合計に変わります。たとえば、$\mathbb{Z}$ にわたるインデックスを持つ平方加算可能な無限次元ベクトルのセットからのベクトルの場合、次の定義が得られます。

 $$(f * g)(i) = \sum_a f(a) g(ia).$$

 2 次元テンソルの場合、$f$ の場合は $(a, b)$ 、$g$ の場合は $(ia, jb)$ というインデックスを持つ対応する合計が得られます。

 $$(f * g)(i, j) = \sum_a\sum_b f(a, b) g(ia, jb).$$ :eqlabel: `eq_2d-conv-discrete`

これは :eqref: `eq_conv-layer`に似ていますが、1 つの大きな違いがあります。 $(i+a, j+b)$ を使用するのではなく、代わりに差分を使用しています。ただし、 :eqref: `eq_conv-layer`と :eqref: `eq_2d-conv-discrete`の間の表記は常に一致させることができるため、この区別はほとんど表面的なものであることに注意してください。 :eqref: `eq_conv-layer`の元の定義は、*相互相関を*より適切に説明しています。これについては次のセクションで改めて説明します。

## チャンネル

:label: `subsec_why-conv-channels`

 Waldo 検出器に戻って、これがどのようなものかを見てみましょう。畳み込み層は、:numref: `fig_waldo_mask`で示されているように、指定されたサイズのウィンドウを選択し、フィルター $\mathsf{V}$ に従って強度を重み付けします。私たちは、「ワルさ」が最も高い場所で隠れ層表現のピークを見つけるようにモデルを学習することを目指すかもしれません。

![](../img/waldo-mask.jpg) :幅: `400px` :ラベル: `fig_waldo_mask`

このアプローチには 1 つだけ問題があります。これまで、画像が赤、緑、青の 3 つのチャネルで構成されていることを幸いにも無視してきました。要するに、画像は 2 次元オブジェクトではなく、高さ、幅、チャネルによって特徴付けられる 3 次のテンソルであり、たとえば形状が $1024 \times 1024 \times 3$ ピクセルです。これらの軸の最初の 2 つは空間的な関係に関係しますが、3 番目の軸は各ピクセル位置に多次元表現を割り当てるものとみなすことができます。したがって、$\mathsf{X}$ を $[\mathsf{X}] *{i, j, k}$ としてインデックス付けします。畳み込みフィルターはそれに応じて適応する必要があります。 $[\mathbf{V}] {a,b}$ の代わりに*、$[\mathsf{V}]_{a,b,c}$ が使用されるようになりました。

さらに、入力が 3 次テンソルで構成されているのと同じように、隠れた表現を 3 次テンソル $\mathsf{H}$ として同様に定式化することが良いことがわかります。言い換えれば、各空間位置に対応する単一の隠れ表現を持つだけではなく、各空間位置に対応する隠れ表現のベクトル全体が必要です。隠された表現は、互いに積み重ねられた多数の 2 次元グリッドで構成されていると考えることができます。入力と同様、これらは*チャネル*と呼ばれることもあります。これらは、学習された特徴の空間化されたセットを後続の層に提供するため、*特徴マップ*と呼ばれることもあります。直感的には、入力に近い下位層では、一部のチャネルがエッジの認識に特化し、他のチャネルがテクスチャを認識できるようになると想像できるかもしれません。

入力 ($\mathsf{X}$) と隠れ表現 ($\mathsf{H}$) の両方で複数のチャネルをサポートするには、$\mathsf{V}$ に 4 番目の座標を追加します。 $[\mathsf{V }]_{a、b、c、d}$。すべてをまとめると次のようになります。

 $$[\mathsf{H}] *{i,j,d} = \sum* {a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \sum_c [\mathsf {V}] *{a, b, c, d} [\mathsf{X}]* {i+a, j+b, c},$$ :eqlabel: `eq_conv-layer-channels`

ここで、$d$ は、隠れた表現 $\mathsf{H}$ の出力チャネルにインデックスを付けます。後続の畳み込み層は、3 次テンソル $\mathsf{H}$ を入力として受け取ります。より一般的には、:eqref: `eq_conv-layer-channels`複数のチャネルの畳み込み層の定義です。$\mathsf{V}$ は層のカーネルまたはフィルターです。

対処しなければならない操作がまだたくさんあります。たとえば、画像内の*どこに*Waldo があるかどうかなど、すべての非表示表現を 1 つの出力に結合する方法を理解する必要があります。また、物事を効率的に計算する方法、複数の層を組み合わせる方法、適切な活性化関数、実際に効果的なネットワークを生み出すための合理的な設計選択を行う方法を決定する必要もあります。この章の残りの部分では、これらの問題について説明します。

## 要約と考察

このセクションでは、第一原理から畳み込みニューラル ネットワークの構造を導き出しました。これが CNN の発明につながったのかどうかは不明ですが、画像処理およびコンピューター ビジョン アルゴリズムが少なくとも低レベルでどのように動作するかに合理的な原則を適用する場合、CNN が*正しい*選択であることを知ることは満足です。特に、画像の変換不変性は、画像のすべてのパッチが同じ方法で処理されることを意味します。局所性とは、対応する隠れた表現を計算するために小さな近傍のピクセルのみが使用されることを意味します。 CNN への最も初期の言及の一部は、Neocognitron の形式です:cite: `Fukushima.1982` 。

私たちが推論する際に遭遇した 2 番目の原則は、少なくともモデルに関する特定の仮定が成立する場合には、関数クラスの表現力を制限することなく、関数クラス内のパラメーターの数を減らす方法です。この制限の結果として複雑さが劇的に軽減され、計算上および統計的に実行不可能な問題が扱いやすいモデルに変わることがわかりました。

チャネルを追加することで、局所性と変換不変性によって畳み込みカーネルに課せられた制限によって失われた複雑性の一部を取り戻すことができました。赤、緑、青以外のチャネルはごく自然に追加されることに注意してください。多くの衛星画像、特に農業や気象学用には数十から数百のチャンネルがあり、代わりにハイパースペクトル画像が生成されます。彼らは多くの異なる波長でデータを報告します。以下では、畳み込みを効果的に使用して画像の次元を操作する方法、位置ベースの表現からチャネルベースの表現に移行する方法、および多数のカテゴリを効率的に処理する方法を見ていきます。

## 演習
1. コンボリューション カーネルのサイズが $\Delta = 0$ であると仮定します。この場合、畳み込みカーネルがチャネルのセットごとに独立して MLP を実装することを示します。これは、ネットワーク アーキテクチャのネットワークにつながります:cite: `Lin.Chen.Yan.2013` 。
1. オーディオ データは多くの場合、1 次元のシーケンスとして表されます。<ol><li>オーディオにローカリティと翻訳の不変性を課す必要があるのはどのような場合ですか?
1. オーディオの畳み込み演算を導出します。
1. コンピュータ ビジョンと同じツールを使用してオーディオを扱うことはできますか?ヒント: スペクトログラムを使用します。

[ディスカッション](https://discuss.d2l.ai/t/64)
